# Modelo para prever o gênero de um filme com base na sua descrição usando o TensorFlow Hub

## Introdução

### TensorFlow Hub

<br>

<div align='justify'>

É uma biblioteca para aprimorar modelos de TensorFlow com aprendizado por transferência. O aprendizado por transferência é o processo de obter os pesos e variáveis de um modelo existente já treinado com muitos dados e usá-lo na sua própria tarefa de dados e previsão.
    
Um dos muitos benefícios do aprendizado por transferência é que você não precisa fornecer tantos dados de treinamento, como se começasse do zero, pois o TensorFlow Hub oferece um repositório completo de pontos de controle já existentes para vários tipos de modelo: imagens, texto e outros.

</div>

### DATASET

<br>

<div align='justify'>
    
Utilizamos no projeto o conjunto de dados de Filmes do Kaggle de domínio público que contém mais de 45.000 filmes. O conjunto tem diversos dados sobre cada filme, mas, para simplificar, vamos usar somente as descrições ("overview") e os gêneros.<a href='https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=movies_metadata.csv'>FONTE</a>

</div>
    

### Importando as bibliotecas

In [3]:
#pip install --upgrade tensorflow-hub

In [20]:
import numpy as np
import pandas as pd

import tensorflow as tf
#import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

from sklearn.preprocessing import MultiLabelBinarizer

In [21]:
data = pd.read_csv('./Downloads/movies_metadata.csv')

In [22]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [23]:
descriptions = data['overview']
genres = data['genres']

In [43]:
classification=genres.to_numpy()
classification

array(["[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",
       ...,
       "[{'id': 28, 'name': 'Action'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]",
       '[]', '[]'], dtype=object)

In [24]:
genres

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [25]:
## Para Simplificar ainda mais, vamos delimitar os gêneros estudados
top_genres = ['Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'Adventure', 'Science Fiction']


> ***VERIFICAR SE NAO CABE O TRAIN SPLIT*** 

In [26]:
## divisão de 80% para treinamento e 20% para teste
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

In [44]:
test_genres.head()


36372    [{'id': 14, 'name': 'Fantasy'}, {'id': 53, 'na...
36373    [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...
36374    [{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...
36375                  [{'id': 99, 'name': 'Documentary'}]
36376                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

### Criação da camada de incorporação com o TF Hub


<div align='justify'>
    
As incorporações de texto criam uma forma de representar partes do texto em espaço de vetor. Portanto, palavras ou frases parecidas ficam mais próximas no espaço de incorporação.

O TF Hub simplifica esse processo disponibilizando incorporações de texto já treinadas com diversos dados de texto. Para texto em inglês, temos:

- Codificador universal de sentenças: para entradas de texto mais longas

- ELMo: incorporações profundas treinadas com o 1B Word Benchmark

- Incorporações do modelo de linguagem de rede neural: treinadas com o Google Notícias

- Word2vec: treinada com a Wikipédia

Estamos definindo um hiperparâmetro do modelo. Por isso, é importante experimentar e ver qual das incorporações tem a maior precisão e indicado que se comece com o modelo que foi treinado com textos mais parecidos ao do projeto em execução. Como nossas descrições de filme são entradas mais longas, imaginamos que conseguiremos a maior precisão com as incorporações do codificador universal de sentenças. Ele codifica as descrições em vetores de texto altamente dimensionais. Note que esse modelo em particular é bem grande e vai ocupar 1 GB.

Podemos usar *hub.text_embedding_column* para criar uma coluna de *feature* para essa camada em uma linha de código, passando a ela o nome da camada ("movie_descriptions") e o URL do modelo do TF Hub que usaremos:
    
</div>

In [27]:

description_embeddings = hub.text_embedding_column(
  "descriptions", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

### Transformação de rótulos em codificações multi-hot


<div align='justify>'>

Como um filme muitas vezes tem diversos gêneros, nosso modelo retornará os vários rótulos possíveis para cada filme. No momento, nossos gêneros são uma lista de strings de cada filme (como ["Action", "Adventure"]). Como todos os rótulos têm que ter o mesmo tamanho, transformaremos essas listas em vetores multi-hot de dígitos 1 e 0, que correspondem aos gêneros presentes em uma determinada descrição. O vetor multi-hot de um filme de ação e aventura (Action e Adventure) ficaria assim:

> Genre lookup, each genre corresponds to an index

> top_genres = ['Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'Adventure', 'Science Fiction']

> Multi-hot label for an action and adventure movie

> [0 0 0 1 0 0 0 1 0]

</div>

Para transformar os rótulos de string em vetores multi-hot em poucas linhas de código, usaremos um utilitário da biblioteca scikit-learn chamado MultiLabelBinarizer:



In [28]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

In [29]:
print(encoder.classes_)

[' ' "'" ',' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' 'A' 'B' 'C' 'D'
 'E' 'F' 'G' 'H' 'M' 'O' 'P' 'R' 'S' 'T' 'V' 'W' '[' ']' 'a' 'b' 'c' 'd'
 'e' 'g' 'h' 'i' 'k' 'l' 'm' 'n' 'o' 'p' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y'
 '{' '}']


In [30]:
print(train_encoded[0])

[1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1
 0 0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1]


### Desenvolvimento e treinamento de um modelo de DNNEstimator


 O primeiro parâmetro que passamos ao DNNEstimator é chamado de "head" (cabeça) e define o tipo de rótulos que nosso modelo deve esperar. Como queremos que o nosso modelo gere diversos rótulos, usaremos "multi_label_head":

In [31]:
multi_label_head = tf.estimator.MultiLabelHead(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

#multi_label_head = tf.estimator.MultiLabelHead(
#    num_classes, weight_column=None, thresholds=None, label_vocabulary=None,
#    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
#)

Agora, podemos passar isso quando instanciarmos o DNNEstimator. O parâmetro "hidden_units" indica quantas camadas teremos na nossa rede. Esse modelo usa 2 camadas: a primeira tem 64 neurônios e a segunda tem 10. O número e o tamanho das camadas é um hiperparâmetro. Portanto, o ideal é experimentar valores diferentes para ver qual funciona melhor com o conjunto de dados. Por fim, passamos nossas colunas de recurso ao Estimator. Nesse caso, só temos uma (a descrição), e já a definimos como uma coluna de incorporação do TF Hub acima para podermos passá-la aqui como uma lista:

In [32]:
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings]
)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/v9/js6pdfh90_10wkfgz737v76w0000gn/T/tmp20dofii7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/v9/js6pdfh90_10wkfgz737v76w0000gn/T/tmp20dofii7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Já está quase tudo pronto para treinar o modelo. Antes de podermos treinar nossa instância do estimador, precisamos definir a função "input" (entrada) do treinamento. Essa função conecta nossos dados ao modelo. Aqui, vamos usar um "numpy_input_fn" e inserir os dados no nosso modelo como matrizes "numpy".

Os parâmetros "batch_size" e "num_epochs" da nossa função "input" são hiperparâmetros. "batch_size" diz ao modelo quantos exemplos serão passados ao modelo em uma iteração e "num_epochs" é o número de vezes que nosso modelo analisará todo o conjunto de treinamento.

In [35]:
# Format our data for the numpy_input_fn
features = {
  "descriptions": np.array(train_descriptions).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=32, 
    num_epochs=20
)

### Treinando o modelo

In [18]:
#features = {
#  "descriptions": np.array(train_descriptions)
#}
#labels = np.array(train_encoded)
#train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=1024, num_epochs=25)
#estimator = tf.estimator.DNNEstimator(
#    head=multi_label_head,
#    hidden_units=[64,10],
#    feature_columns=[description_embeddings])

In [36]:
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 22733 into /var/folders/v9/js6pdfh90_10wkfgz737v76w0000gn/T/tmp20dofii7/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 22733...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 22733...


INFO:tensorflow:Loss for final step: 0.40297055.


INFO:tensorflow:Loss for final step: 0.40297055.


Para avaliar a precisão do nosso modelo, criamos um eval "input_function" com os dados do teste e chamamos "estimator.evaluate()":

In [38]:

eval_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)},
                                                   test_encoded.astype(np.int32), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 22733: auc = 0.8920273, auc_precision_recall = 0.8399027, average_loss = 0.4322122, global_step = 22733, loss = 0.4331238


INFO:tensorflow:Saving dict for global step 22733: auc = 0.8920273, auc_precision_recall = 0.8399027, average_loss = 0.4322122, global_step = 22733, loss = 0.4331238


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22733: /var/folders/v9/js6pdfh90_10wkfgz737v76w0000gn/T/tmp20dofii7/model.ckpt-22733


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22733: /var/folders/v9/js6pdfh90_10wkfgz737v76w0000gn/T/tmp20dofii7/model.ckpt-22733


{'auc': 0.8920273,
 'auc_precision_recall': 0.8399027,
 'average_loss': 0.4322122,
 'loss': 0.4331238,
 'global_step': 22733}

Esse modelo atingiu 89,2% de área abaixo da curva (AUC) e 84% de precisão/recall. Com 43,22% de perda.

### Geração de previsões no modelo treinado


Chegamos na melhor parte: gerar previsões com dados que o nosso modelo nunca viu. Primeiro, vamos criar uma matriz com algumas descrições (peguei as descrições abaixo no IMDB):

In [39]:
raw_test = [
    "An examination of our dietary choices and the food we put in our bodies. Based on Jonathan Safran Foer's memoir.", # Documentary
    "A teenager tries to survive the last week of her disastrous eighth-grade year before leaving to start high school.", # Comedy
    "Ethan Hunt and his IMF team, along with some familiar allies, race against time after a mission gone wrong." # Action, Adventure
]

Em seguida, vamos definir nossa função "input" de previsão e chamar "predict()":

In [41]:
predict_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn({"descriptions": np.array(raw_test).astype(np.str)},
                                                      shuffle=False)

results = estimator.predict(predict_input_fn)




Por fim, podemos iterar os resultados e exibir os 2 principais gêneros encontrados para cada filme junto com os valores de confiança:

In [42]:
for movie_genres in results:
  top_2 = movie_genres['probabilities'].argsort()[-2:][::-1]
  for genre in top_2:
    text_genre = encoder.classes_[genre]
    print(text_genre + ': ' + str(round(movie_genres['probabilities'][genre] * 100, 2)) + '%')
    
    
    


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


e: 89.26%
,: 88.86%
e: 91.13%
,: 90.45%
e: 92.33%
,: 91.76%


Nosso modelo consegue marcar todas as descrições de filme acima corretamente.